# Comparing Pancakeswap GetReserves() Speed

This notebook compares the speed of `GetReserves()` between bloXroute cloud API and standard fullnodes.

Start a fullnode without a local gateway, and run [fullnode_pair_reserve](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/fullnode/pair_reserve/fullnode_pair_reserve.go) and [bloxroute_pair_reserve](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/bloxroute/pair_reserve/bloxroute_pair_reserve.go) commands to collect data.

Note: Make sure the BSC fullnode is fully synced before this benchmark, type `eth.syncing` in geth console and make sure it is `faluse`, which means the fullnode is synced.

Hardware:

* A machine with 128 cores, 256GB memory in Fremont IDC, running a BSC fullnode without a local bloXroute gateway
* A `m5zn.3xlarge` instance in AWS Virginia regon, running a BSC fullnode without a local bloXroute gateway

In [1]:
import gzip
import json
import pandas as pd
from typing import Dict

In [2]:
pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

In [3]:
def read_timestamp(file: str) -> Dict[int, int]:
    result: Dict[str, int] = {}  # hash -> received_at
    f_in = gzip.open(file, "rt") if file.endswith('.json.gz') else open(file, "rt")
    with f_in:
        for line in f_in:
            obj = json.loads(line)
            hash_code = hash(obj['pair'] + obj['reserve0'] + obj['reserve1'] + str(obj['block_number']))
            result[hash_code] = obj['received_at']
    return result

In [4]:
def compare_timestamp(file1: str, file2: str) -> pd.DataFrame:
    dict1 = read_timestamp(file1)
    dict2 = read_timestamp(file2)
    diff = {}
    for hash_code, received_at in dict1.items():
        if hash_code in dict2:
            diff[hash_code] = received_at-dict2[hash_code]
    
    df = pd.DataFrame(diff.items(), columns=['hash_code', 'gap'])
    # remove outliers
    quantile05 = df['gap'].quantile(0.05)
    quantile95 = df['gap'].quantile(0.95)
    return df[(df['gap'] >= quantile05) & (df['gap']<=quantile95)]

## 1. Compare the fullnode and Cloud API in Fremont

The is a fullnode in Fremont IDC, and `fullnode_pair_reserve` pulls data from the fullnode periodically, and `bloxroute_pair_reserve` subscribes to the `ethOnBlock` stream from Fremont.

In [5]:
compare_timestamp('./data/bloxroute-pair-reserve-cloud-fremont.json.gz', './data/fullnode-pair-reserve-fremont.json.gz').describe()

,hash_code,gap
count,2.689400e+04,26894.000000
mean,7.723354e+14,-479.474492
std,5.309797e+18,332.465344
min,-9.223323e+18,-1368.000000
25%,-4.615890e+18,-669.000000
50%,2.088149e+16,-551.000000
75%,4.573181e+18,-367.000000
max,9.223147e+18,322.000000


**Conclusion: the cloud API in 551ms faster than the local fullnode in Fremont**

## 2. Compare the fullnode and Cloud API in AWS Virginia

The is a fullnode in AWS Virginia region, and `fullnode_pair_reserve` pulls data from the fullnode periodically, and `bloxroute_pair_reserve` subscribes to the `ethOnBlock` stream from Fremont.

In [6]:
compare_timestamp('./data/bloxroute-pair-reserve-cloud-virginia.json.gz', './data/fullnode-pair-reserve-virginia.json.gz').describe()

,hash_code,gap
count,2.754000e+04,27540.000000
mean,2.336288e+15,-639.175563
std,5.312855e+18,824.880215
min,-9.223323e+18,-5119.000000
25%,-4.616191e+18,-893.000000
50%,6.614838e+15,-331.000000
75%,4.603770e+18,-120.000000
max,9.223147e+18,428.000000


**Conclusion: the cloud API in 331ms faster than the local fullnode in Fremont**

## 3. Accessing cloud APIs from different locations

In [7]:
compare_timestamp('./data/bloxroute-pair-reserve-cloud-virginia.json.gz', './data/bloxroute-pair-reserve-cloud-fremont.json.gz').describe()

,hash_code,gap
count,2.640900e+04,26409.000000
mean,-7.243110e+15,-251.515430
std,5.313338e+18,161.963195
min,-9.223323e+18,-424.000000
25%,-4.636276e+18,-336.000000
50%,1.651374e+16,-297.000000
75%,4.578457e+18,-236.000000
max,9.223147e+18,686.000000


**Conclusion: Accessing the cloud API from AWS Virginia is 297ms faster from Fremont, because the bloXroute websocket server `wss://virginia.bsc.blxrbdn.com/ws` is in AWS Virginia region**

## 4. Compare two fullnodes

In [8]:
compare_timestamp('./data/fullnode-pair-reserve-virginia.json.gz', './data/fullnode-pair-reserve-fremont.json.gz').describe()

,hash_code,gap
count,2.804200e+04,28042.000000
mean,2.650828e+15,-60.257756
std,5.322216e+18,853.672150
min,-9.223323e+18,-953.000000
25%,-4.627704e+18,-674.000000
50%,4.498959e+15,-263.000000
75%,4.611167e+18,226.000000
max,9.223147e+18,4288.000000


**Conclusion: The fullnode in AWS Virginia is 263ms faster than the fullnode in Fremont, because many BSC validator nodes are running in AWS Virginia region, therefore the fullnode in AWS Virginia syncs faster than the fullnode in Fremont**